In [ ]:
from torch.utils.data import DataLoader
import sys 
import torch
from torch import einsum
from torch import nn
sys.path.append("/mnt/DATA-4/hx/Ruipath/MunchkinCat")
sys.path.append("/mnt/DATA-4/hx/Ruipath/scFoundation/model/")
# from model_finetune import *
from data_loader import *
from models import *

/mnt/DATA-4/hx/conda_envs/UNI/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# slide_ids, slide_patch_ids = Get_hest_meta()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
dataset = HESTDataset(
    data_root="/mnt/DATA-4/hx/Ruipath/hest_data"
)

In [4]:
local_dir = "/mnt/DATA-4/hx/Ruipath/RuiPathViT/Ruipath_visionfoundation_v1.bin"
ruipathvit = RuiPathViT(local_dir, device)
ruipathvit.build()

In [ ]:
ckpt_path = "/mnt/DATA-4/hx/Ruipath/scFoundation/model/models/models.ckpt"
out_dim = ruipathvit.model.num_features
sc_model = scFoundation(ckpt_path, out_dim, device)

In [57]:
import torch.optim as optim

train_loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=0)   
model = RuiPathST(ruipathvit, sc_model).cuda()

# 使用自定义 criterion（这里不需要 nn.CrossEntropyLoss）
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(3):
    model.train()
    running_loss = 0.0
    for i, (images, genes) in enumerate(train_loader):  # 假设 dataloader 返回图像+基因
        images = images.cuda()
        genes = genes.cuda()

        optimizer.zero_grad()
        loss = model(images, genes)  # 假设模型这样设计
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        num_batches = i + 1
        avg_loss = running_loss / num_batches
        if i % 3 == 0:
            print(f'Epoch [{epoch+1}/{3}], Step [{i+1}/{len(train_loader)}], \
                  Total Loss: {running_loss:.4f}, Avg Loss: {avg_loss:.4f}')

    # 验证部分（可选，InfoNCE 通常不直接验证 accuracy）
    print(f"Loss on epoch {epoch+1}: {running_loss / len(train_loader):.4f}")

torch.save(model.state_dict(), "/mnt/DATA-4/hx/Ruipath/RuiPathST_ckp")
print(f"Model saved to /mnt/DATA-4/hx/Ruipath/RuiPathST_ckp")

Epoch [1/3], Step [1/3157],                   Total Loss: 0.0351, Avg Loss: 3.5122
Epoch [1/3], Step [4/3157],                   Total Loss: 0.4263, Avg Loss: 10.6580
Epoch [1/3], Step [7/3157],                   Total Loss: 0.7644, Avg Loss: 10.9200
Epoch [1/3], Step [10/3157],                   Total Loss: 1.1432, Avg Loss: 11.4322
Epoch [1/3], Step [13/3157],                   Total Loss: 1.2732, Avg Loss: 9.7938
Epoch [1/3], Step [16/3157],                   Total Loss: 1.3912, Avg Loss: 8.6950
Epoch [1/3], Step [19/3157],                   Total Loss: 1.5028, Avg Loss: 7.9094
Epoch [1/3], Step [22/3157],                   Total Loss: 1.6193, Avg Loss: 7.3606
Epoch [1/3], Step [25/3157],                   Total Loss: 1.7420, Avg Loss: 6.9681
Epoch [1/3], Step [28/3157],                   Total Loss: 1.8553, Avg Loss: 6.6261
Epoch [1/3], Step [31/3157],                   Total Loss: 1.9865, Avg Loss: 6.4079
Epoch [1/3], Step [34/3157],                   Total Loss: 2.0981, Avg Loss:

KeyboardInterrupt: 